## Model Selection

<a id='up'></a>

 Classification trees are appropriate for this problem, as they successively determine decision criteria based on subsets of the initial variables. It corresponds to an intuitive representation of the consumers, each one being associated with a cluster linked to its credit prole. We chose to use three different models: 

In this chapter, we choose the best model and the best parameters to predict the number of visitors.

* 0. [Load libraries](#load-libraries)
* 1. [Data import](#data-import)
* 2. [Model evaluation - hyperparameters](#hyperparameters)
* 3. [Finding p-value](#pval)
* 4. [Imbalance correction](#imbalance-correction)
* 4.1. [RandomForestClassifier](#rfc)
* 5. [Models](#models)
* 5.1 [Fit Models](#fit-model)
* 5.2 [predict Models on train](#pmtr)
* 5.3 [RSMLE - Root Mean Squared Logarithmic Error](#rsmle)
* 6. [Predict Test](#test)
* 7. [Sub](#sub)

### <a id='load-libraries'>0. Load libraries</a>

In [1]:
# Load libraries
import numpy as np 
import pandas as pd 
from subprocess import check_output
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import glob, re
from sklearn import *
from datetime import datetime
from xgboost import XGBRegressor

In [3]:
np.random.seed(10)

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [4]:
#from importlib import reload
#import pyMechkar as mechkar
#reload(mechkar)

In [5]:
#from collections import Counter
#from sklearn.datasets import make_classification
#from imblearn.over_sampling import SMOTE

In [6]:
# Install a pip package in the current Jupyter kernel
#import sys
#!{sys.executable} -m pip install tableone

In [7]:
#import tableone as TableOne

In [8]:
from sklearn.ensemble import RandomForestClassifier # Random foreset section 4 (Imbalance correction)
from sklearn import utils # for section 4 in order to avoid ValueError: Unknown label type: 'continuous'
from sklearn.metrics import accuracy_score #
from sklearn.metrics import roc_auc_score


[Up to the header](#up)

### <a id='data-import'>1.Data import</a> 

In [9]:
# Data Aggregation
train = pd.read_csv('./input/train.csv')
test = pd.read_csv('./input/test.csv')

In [10]:
train.head(1)

,Unnamed: 0,Unnamed: 0.1,visit_date,visitors,air_store_id,latitude,longitude,month,date,dw,...,Ōsaka-fu,Hyōgo-ken,Hokkaidō,Shizuoka-ken,Fukuoka-ken,Hiroshima-ken,Niigata-ken,Miyagi-ken,reserve_visitors_air_1,air_date_diff_1
0,0,0,2016-01-13,25,air_ba937bf13d40fb24,35.658068,139.751599,1,13,2,...,0,0,0,0,0,0,0,0,NaN,NaN


In [11]:
test.head(1)

,Unnamed: 0,id,visitors,visit_date,air_store_id,dw,dy,date,month,holiday_flg,...,Saturday,Tōkyō-to,Ōsaka-fu,Hyōgo-ken,Hiroshima-ken,Fukuoka-ken,Hokkaidō,Miyagi-ken,Niigata-ken,Shizuoka-ken
0,0,air_00a91d42b08b08d9_2017-04-23,0,2017-04-23,air_00a91d42b08b08d9,6,2017,23,4,0,...,0,1,0,0,0,0,0,0,0,0


In [12]:
train = train.drop(['Unnamed: 0' , 'Unnamed: 0.1'], axis=1)
test = test.drop(['Unnamed: 0'], axis=1)

[Up to the header](#up)

### <a id='hyperparameters'>2. Model evaluation - hyperparameters</a>

In [13]:
col = [c for c in train if c not in ['id', 'air_store_id', 'visit_date','visitors']] 

###### For missing values

In [14]:
train = train.fillna(-1)
test = test.fillna(-1)

In [15]:
#for c, dtype in zip(train.columns, train.dtypes):
#    if dtype == np.float64:
#        train[c] = train[c].astype(np.float32)

#for c, dtype in zip(test.columns, test.dtypes):
#    if dtype == np.float64:
 #       test[c] = test[c].astype(np.float32)

### RSMLE - Root Mean Squared Logarithmic Error

In [16]:
def RMSLE(y, pred):
    return metrics.mean_squared_error(y, pred)**0.5

In [17]:
X=train[col]
X_train = train[train.visit_date<'2017-03-01'][col]
X_test = train[train.visit_date>'2017-03-01'][col]

y_train = np.log1p(train[train.visit_date<'2017-03-01']['visitors'].values)
y_test = np.log1p(train[train.visit_date>'2017-03-01']['visitors'].values)

In [18]:
X1_train = X_train.copy()
X1_test = X_test.copy()
y1_train =y_train.copy()
y1_test = y_test.copy()

In [19]:
X1_train['visitors'] = y1_train

[Up to the header](#up)

#### <a id='pval'>3. Finding p-value</a>

In [20]:
#cat = ['Dining bar', 'Izakaya', 'Other', 'Italian/French','Cafe/Sweets',\
#      'Japanese food', 'Bar/Cocktail', 'Creative cuisine','Western food',\
#     'Yakiniku/Korean food', 'Asian','International cuisine','Okonomiyaki/Monja/Teppanyaki',\
#    'Karaoke/Party','Wednesday', 'Thursday', 'Friday', 'Saturday','Monday',\
#     'Tuesday', 'Sunday','Tōkyō-to','Ōsaka-fu','Hyōgo-ken','Hokkaidō',\
#       'Shizuoka-ken', 'Fukuoka-ken', 'Hiroshima-ken','Niigata-ken', 'Miyagi-ken'\
#       'sunday','saturday','holiday_flg']

# non-normal variables
#nonnormal = ['visitors']


#ableOne(data, columns, categorical, groupby, nonnormal, label_suffix=True, pval = True)
# create tableone with the input arguments
#mytable = TableOne(data=X1_train,columns=col, categorical=cat,nonnormal=nonnormal,pval = True,label_suffix=True)
#mytable

#overall_table = TableOne(train, label_suffix=True)
#X1_train.dtypes[X1_train.dtypes=='float64']

In [21]:
#mtrain3= mechkar.pyMechkar().Table1(data=X1_train, y='visitors')
#mtrain3
#mtest = mechkar.pyMechkar().exploreData(data=X1_test, y=y1_test)

[Up to the header](#up)

### <a id='imbalance-correction'>4. Imbalance correction</a>

##### Use Tree-Based Algorithms
Decision trees often perform well on imbalanced datasets,<br> 
because their hierarchical structure allows them to learn signals from both classes.<br><br>

In modern applied machine learning, tree ensembles (Random Forests, Gradient Boosted Trees, etc.)<br>
almost always outperform singular decision trees

```python
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score 
from sklearn.metrics import roc_auc_score
```

In [22]:
Xtra = X1_train.copy()
ytra = y1_train.copy()

LabelEncoder - convert string or float values to 0 .. n classes.

If we put as input <code>Xtra</code>, <code>ytra</code> to fit method it cause error. To avoid it we will convert and encode labels.

In [24]:
lab_enc = preprocessing.LabelEncoder()
training_scores_encoded = lab_enc.fit_transform(ytra)
print(training_scores_encoded)
print(utils.multiclass.type_of_target(ytra))
print(utils.multiclass.type_of_target(ytra.astype('int')))
print(utils.multiclass.type_of_target(training_scores_encoded))


[24 20 39 ... 14 21  2]
continuous
multiclass
multiclass


### <a id='rfc'>4.1 RandomForestClassifier</a>

In [25]:
# Separate input features (Xtra) and target variable (ytra)
#Xtra = Xtra.drop('visitors', axis=1)

# Train model
clf = RandomForestClassifier()
clf.fit(Xtra, training_scores_encoded)
print("RandomForestClassifier")

# Predict on training set
pred_y = clf.predict(Xtra)

# Is our model still predicting just one class?
print(np.unique(pred_y))

# The accuracy 51%
print(accuracy_score(training_scores_encoded, pred_y)) #0.5102562512817334

# The AUROC
prob_y = clf.predict_proba(Xtra)
prob_y = [p[1] for p in prob_y]
print(roc_auc_score(training_scores_encoded,prob_y) )


C:\Users\gali\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier
[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 130 131 132 133 134 135 136 137 139 140 141 142 144 147 148
 150 151 153 154 155 156 158 159 160 161 162 163 164 165 166 167 168 170
 172 174 176 180 183 184 185 188]
0.5102562512817334


ValueError: multiclass format is not supported

[Up to the header](#up)

## <a id='models'>5. Models </a>

##### Model GradientBoostingRegressor

In [17]:
model1 = ensemble.GradientBoostingRegressor(learning_rate=0.2, random_state=3, n_estimators=200, subsample=0.8, 
                      max_depth =10)

##### Model KNeighborsRegressor

In [18]:
model2 = neighbors.KNeighborsRegressor(n_jobs=-1, n_neighbors=4)

##### Model XGBRegressor

In [19]:
model3 = XGBRegressor(learning_rate=0.2, n_estimators=280, subsample=0.8, 
                      colsample_bytree=0.8, max_depth =12)
#random_state=3

[Up to the header](#up)

### <a id='fit-model'>5.1 Fit Models</a>

##### fit model GradientBoostingRegressor

In [20]:
model1.fit(train[col], np.log1p(train['visitors'].values))

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.2, loss='ls', max_depth=10, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=200, n_iter_no_change=None, presort='auto',
             random_state=3, subsample=0.8, tol=0.0001,
             validation_fraction=0.1, verbose=0, warm_start=False)

##### fit model KNeighborsRegressor

In [21]:
model2.fit(train[col], np.log1p(train['visitors'].values))

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=-1, n_neighbors=4, p=2,
          weights='uniform')

##### fit model XGBRegressor

In [22]:
model3.fit(train[col], np.log1p(train['visitors'].values))

XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
       gamma=0, learning_rate=0.2, max_delta_step=0, max_depth=12,
       min_child_weight=1, missing=None, n_estimators=280, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=0.8)

[Up to the header](#up)

### <a id='pmtr'>5.2 predict Models on train</a>

#####  predict model GradientBoostingRegressor on train

In [23]:
preds1 = model1.predict(train[col])

#####  predict model KNeighborsRegressor on train

In [24]:
preds2 = model2.predict(train[col])

#####  predict model XGBRegressor on train

In [25]:
preds3 = model3.predict(train[col])

[Up to the header](#up)

 ### <a id='rsmle'>RSMLE - Root Mean Squared Logarithmic Error</a>

In [26]:
print('RMSE GradientBoostingRegressor: ', RMSLE(np.log1p(train['visitors'].values), preds1))

('RMSE GradientBoostingRegressor: ', 0.6043729591762557)


In [27]:
print('RMSE KNeighborsRegressor: ', RMSLE(np.log1p(train['visitors'].values), preds2))

('RMSE KNeighborsRegressor: ', 0.6657030733438531)


In [28]:
print('RMSE XGBRegressor: ', RMSLE(np.log1p(train['visitors'].values), preds3))

('RMSE XGBRegressor: ', 0.5748429053017586)


[Up to the header](#up)

### <a id='test'>6. Predict Test</a>

#### predict GradientBoostingRegressor

In [29]:
preds1 = model1.predict(test[col])

#### predict KNeighborsRegressor

In [30]:
preds2 = model2.predict(test[col])

#### predict XGBRegressor

In [31]:
preds3 = model3.predict(test[col])

In [32]:
test['visitors'] = 0.3*preds1+0.3*preds2+0.4*preds3
test['visitors'] = np.expm1(test['visitors']).clip(lower=0.)

[Up to the header](#up)

## <a id='sub'>7. Sub</a>

In [33]:
sub = test[['id','visitors']].copy()

In [34]:
sub.head()

,id,visitors
0,air_00a91d42b08b08d9_2017-04-23,9.405370
1,air_00a91d42b08b08d9_2017-04-24,12.509069
2,air_00a91d42b08b08d9_2017-04-25,19.572412
3,air_00a91d42b08b08d9_2017-04-26,20.436537
4,air_00a91d42b08b08d9_2017-04-27,22.645801


In [36]:
sub1.to_csv(r'C:\Users\sergey\Documents\Recruit Restaurant Visitor_2\submission.csv')

[Up to the header](#up)